In [1]:
import os
import re
import json
import math
import numpy as np
from tqdm import tqdm_notebook as tqdm

from keras_bert import load_vocabulary, load_trained_model_from_checkpoint, Tokenizer, get_checkpoint_paths

import keras.backend as K
from keras.layers import Input, Dense, Lambda, Multiply, Masking, Concatenate
from keras.models import Model
from keras.preprocessing.sequence import pad_sequences
from keras.callbacks import Callback, ModelCheckpoint
from keras.utils.data_utils import Sequence
from keras.utils import multi_gpu_model

from nl2sql.utils import read_data, read_tables, SQL, MultiSentenceTokenizer, Query, Question, Table
from nl2sql.utils.optimizer import RAdam


# config
os.environ['CUDA_VISIBLE_DEVICES'] = '' 
NUM_GPUS = len(os.environ['CUDA_VISIBLE_DEVICES'].split(','))


Using TensorFlow backend.


In [2]:
## model1

def get_model(num_sel_agg,num_cond_op,num_cond_conn_op):

    bert_model = load_trained_model_from_checkpoint(paths.config, paths.checkpoint, seq_len=None)
    for l in bert_model.layers:
        l.trainable = True
        
    inp_token_ids = Input(shape=(None,), name='input_token_ids', dtype='int32')
    inp_segment_ids = Input(shape=(None,), name='input_segment_ids', dtype='int32')
    inp_header_ids = Input(shape=(None,), name='input_header_ids', dtype='int32')
    inp_header_mask = Input(shape=(None, ), name='input_header_mask')

    x = bert_model([inp_token_ids, inp_segment_ids]) # (None, seq_len, 768)

    # predict cond_conn_op
    x_for_cond_conn_op = Lambda(lambda x: x[:, 0])(x) # (None, 768)
    p_cond_conn_op = Dense(num_cond_conn_op, activation='softmax', name='output_cond_conn_op')(x_for_cond_conn_op)

    # predict sel_agg
    x_for_header = Lambda(seq_gather, name='header_seq_gather')([x, inp_header_ids]) # (None, h_len, 768)
    header_mask = Lambda(lambda x: K.expand_dims(x, axis=-1))(inp_header_mask) # (None, h_len, 1)

    x_for_header = Multiply()([x_for_header, header_mask])
    x_for_header = Masking()(x_for_header)

    p_sel_agg = Dense(num_sel_agg, activation='softmax', name='output_sel_agg')(x_for_header)

    x_for_cond_op = Concatenate(axis=-1)([x_for_header, p_sel_agg])
    p_cond_op = Dense(num_cond_op, activation='softmax', name='output_cond_op')(x_for_cond_op)


    model = Model(
        [inp_token_ids, inp_segment_ids, inp_header_ids, inp_header_mask],
        [p_cond_conn_op, p_sel_agg, p_cond_op]
    )

    return model

def seq_gather(x):
    seq, idxs = x
    idxs = K.cast(idxs, 'int32')
    return K.tf.batch_gather(seq, idxs)

bert_model_path = '../../../experiments/model/chinese_wwm_L-12_H-768_A-12'
paths = get_checkpoint_paths(bert_model_path)

model_path = '../model/m1_ep10.h5'

# init model
num_sel_agg = len(SQL.agg_sql_dict) + 1
num_cond_op = len(SQL.op_sql_dict) + 1
num_cond_conn_op = len(SQL.conn_sql_dict)

model = get_model(num_sel_agg,num_cond_op,num_cond_conn_op)

model.load_weights(model_path)





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.







Instructions for updating:
`tf.batch_gather` is deprecated, please use `tf.gather` with `batch_dims=-1` instead.


In [5]:
# model.input[2].name

input_dict = dict()

for input_ in model.input:
    input_dict[input_.name]=input_

input_dict


{'input_header_ids:0': <tf.Tensor 'input_header_ids:0' shape=(?, ?) dtype=int32>,
 'input_header_mask:0': <tf.Tensor 'input_header_mask:0' shape=(?, ?) dtype=float32>,
 'input_segment_ids:0': <tf.Tensor 'input_segment_ids:0' shape=(?, ?) dtype=int32>,
 'input_token_ids:0': <tf.Tensor 'input_token_ids:0' shape=(?, ?) dtype=int32>}

In [9]:
output_dict = dict()

for output_ in model.output:
    output_dict[output_.name]=output_

output_dict

{'output_cond_conn_op/Softmax:0': <tf.Tensor 'output_cond_conn_op/Softmax:0' shape=(?, 3) dtype=float32>,
 'output_cond_op/truediv:0': <tf.Tensor 'output_cond_op/truediv:0' shape=(?, ?, 5) dtype=float32>,
 'output_sel_agg/truediv:0': <tf.Tensor 'output_sel_agg/truediv:0' shape=(?, ?, 7) dtype=float32>}

In [7]:
import tensorflow as tf
from tensorflow.python.util import compat

def export_savedmodel(model,output_path):
    '''
    传入keras model会自动保存为pb格式
    '''
    model_path = output_path # 模型保存的路径
    model_version = 0 # 模型保存的版本
    
    # 从网络的输入输出创建预测的签名
    input_dict = dict()
    for input_ in model.input:
        input_dict[input_.name]=input_

    output_dict = dict()
    for output_ in model.output:
        output_dict[output_.name]=output_
    
    model_signature = tf.saved_model.signature_def_utils.predict_signature_def(
        inputs=input_dict, outputs=output_dict
    )
    # 使用utf-8编码将 字节或Unicode 转换为字节
    export_path = os.path.join(compat.as_bytes(model_path), compat.as_bytes(str(model_version))) # 将保存路径和版本号join
    builder = tf.saved_model.builder.SavedModelBuilder(export_path) # 生成"savedmodel"协议缓冲区并保存变量和模型
    builder.add_meta_graph_and_variables( # 将当前元图添加到savedmodel并保存变量
    sess=K.get_session(), # 返回一个 session 默认返回tf的sess,否则返回keras的sess,两者都没有将创建一个全新的sess返回
    tags=[tf.saved_model.tag_constants.SERVING], # 导出模型tag为SERVING(其他可选TRAINING,EVAL,GPU,TPU)
    clear_devices=True, # 清除设备信息
    signature_def_map={ # 签名定义映射
        tf.saved_model.signature_constants.DEFAULT_SERVING_SIGNATURE_DEF_KEY: # 默认服务签名定义密钥
        model_signature # 网络的输入输出策创建预测的签名
    })
    builder.save() # 将"savedmodel"协议缓冲区写入磁盘.
    print("save model pb success ...")

output_path = '../model/m1_ep10'
export_savedmodel(model, output_path) # 将模型传入保存模型的方法内,模型保存成功.

INFO:tensorflow:No assets to save.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../model/m1_ep10/0/saved_model.pb
save model pb success ...
